#Two-Stages Deep Learning Project
<p>This project has been developped by the Montreal Hacknight community.<p>

<p><ul>https://github.com/robintibor/braindecode</ul>
<ul>https://github.com/NeuroTechX/moabb</ul></p>

<p>This project wouldn't have been possible without the contribution of these people: Arna (...), Justin (...), Yannick Roy, Eamon Egan, (...) and it is currently being supported by Fred Simard (fs@re-ak.com)</p>

<p>This project consist in deriving a two-stages training process from the brain decode implementation of a Shallow Convolutional Network. The core motivation is to combine the strenghts of deep learning - that is being one of the most powerful ML algorithms out-there - while grinding down it's fall back - it requires an immense of data.</p>

<p>The goal is to develop a method to train the network in two stages:</p>

<p><ul>1) Train the network over a large dataset, this training forms some kind of prior over the network (all but one subject)</ul>
<ul>2) Train the network a second time, on the dataset of interest (the subject left aside)</ul></p>

<p>The premise motivating this approach is that the second training is likely to proceed faster than what is normally required to train a deep net, without sacrificing the performance, and will enable the possibility to train a deep learner on a smaller dataset; further, we suspect that the second training will be able to exploit the specificities of the dataset of interest in a way to surpasses the first training, which was trained on a generalistic dataset.</p>

In [1]:


"""
#
# Download the dataset and segment per subject, 
# NOTE: this block should be executed once and once only
#
"""
import os
if not os.path.exists("bnci-2014"):
  os.mkdir("bnci-2014")

#if not os.path.exists("bnci-2014/S01E.mat"):
if True:
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S02T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S02E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S03T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S03E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S04T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S04E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S05T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S05E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S06T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S06E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S07T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S07E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S08T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S08E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S09T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S09E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S10T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S10E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S11T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S11E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S12T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S12E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S13T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S13E.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S14T.mat -P "bnci-2014"
  !wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S14E.mat -P "bnci-2014"
  
  
#note just in case, there are duplicates
!rm bnci-2014/*.mat.*

--2019-01-18 00:57:08--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01E.mat
Resolving bnci-horizon-2020.eu (bnci-horizon-2020.eu)... 91.227.204.35
Connecting to bnci-horizon-2020.eu (bnci-horizon-2020.eu)|91.227.204.35|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://lampx.tugraz.at/~bci/database/002-2014/S01E.mat [following]
--2019-01-18 00:57:08--  https://lampx.tugraz.at/~bci/database/002-2014/S01E.mat
Resolving lampx.tugraz.at (lampx.tugraz.at)... 129.27.124.207
Connecting to lampx.tugraz.at (lampx.tugraz.at)|129.27.124.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23769588 (23M)
Saving to: ‘bnci-2014/S01E.mat’

S01E.mat            100%[===================>]  22.67M  6.59MB/s    in 5.8s    

2019-01-18 00:57:15 (3.92 MB/s) - ‘bnci-2014/S01E.mat’ saved [23769588/23769588]

--2019-01-18 00:57:15--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat
Resolving bnci-horizon-2020.eu (bnci-hor

In [0]:
# Installation
# NOTE: Should only be called once

#install brain decode
!pip install braindecode -q

# install pytorch
# ref: http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

#!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
#  cu91/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
import torch



In [52]:

import numpy as np
import scipy.io as sio

DATASET_PATH = "bnci-2014"

TEST_SUBJECT_IDX = np.array([0]) # needs be
VEHICULE_SUBJECTS_IDX = np.array([1,2]) # needs be
TO_REMOVE = np.r_[TEST_SUBJECT_IDX,VEHICULE_SUBJECTS_IDX]

TEST_SUBJECT_IDX *= 2
VEHICULE_SUBJECTS_IDX *= 2
TO_REMOVE *= 2

TRIAL_WINDOW_LENGTH = 7

allSubjects = os.listdir(DATASET_PATH)
allSubjects.sort()

testSubject = []
vehiculeSubject = []

TO_REMOVE[::-1].sort()


for i in range(TEST_SUBJECT_IDX.shape[0]):
    testSubject.append(allSubjects[TEST_SUBJECT_IDX[i]])
    testSubject.append(allSubjects[TEST_SUBJECT_IDX[i]+1])
    
for i in range(VEHICULE_SUBJECTS_IDX.shape[0]):
    vehiculeSubject.append(allSubjects[VEHICULE_SUBJECTS_IDX[i]])
    vehiculeSubject.append(allSubjects[VEHICULE_SUBJECTS_IDX[i]+1])
    
testSubject.sort()
vehiculeSubject.sort()


for i in range(TO_REMOVE.shape[0]):
    # double pop to remove *T and *E
    allSubjects.pop(TO_REMOVE[i])
    allSubjects.pop(TO_REMOVE[i]) 

print("Test subject: " + repr(testSubject))
print("Vehicule subjects: " + repr(vehiculeSubject))
print("Train subjects: " + repr(allSubjects))

X1 = []
y1 = []


for f in allSubjects:
    # read file
    d1T = sio.loadmat(DATASET_PATH + "/" + f)
    
    samplingRate = d1T['data'][0][0][0][0][3][0][0]
    trialLength = TRIAL_WINDOW_LENGTH*samplingRate

    # run through all training runs
    for run in range(len(d1T['data'][0])):
        y1.append(d1T['data'][0][run][0][0][2][0]) # labels
        timestamps = d1T['data'][0][run][0][0][1][0] # timestamps
        rawData = d1T['data'][0][run][0][0][0].transpose() # chan x data

        # parse out data based on timestamps
        for start in timestamps:
            end = start + trialLength
            X1.append(rawData[:,start:end]) #15 x 2560

    del rawData
    del d1T

# arrange data into num7py arrays
# also torch expect float32 for samples
# and int64 for labels {0,1}
X1 = np.array(X1).astype(np.float32)
y1 = (np.array(y1).flatten()-1).astype(np.int64)

# rand permute dataset
idx = np.random.permutation(X1.shape[0])
X1 = X1[idx,:,:]
y1 = y1[idx]


X2 = []
y2 = []

for f in testSubject:
    # read file
    d1T = sio.loadmat(DATASET_PATH + "/" + f)
    
    samplingRate = d1T['data'][0][0][0][0][3][0][0]
    trialLength = TRIAL_WINDOW_LENGTH*samplingRate

    # run through all training runs
    for run in range(len(d1T['data'][0])):
        y2.append(d1T['data'][0][run][0][0][2][0]) # labels
        timestamps = d1T['data'][0][run][0][0][1][0] # timestamps
        rawData = d1T['data'][0][run][0][0][0].transpose() # chan x data

        # parse out data based on timestamps
        for start in timestamps:
            end = start + trialLength
            X2.append(rawData[:,start:end]) #15 x 2560

    del rawData
    del d1T

X2 = np.array(X2).astype(np.float32)
y2 = (np.array(y2).flatten()-1).astype(np.int64)


# rand permute dataset
idx = np.random.permutation(X2.shape[0])

X2 = X2[idx,:,:]
y2 = y2[idx]



X3 = []
y3 = []

for f in testSubject:
    # read file
    d1T = sio.loadmat(DATASET_PATH + "/" + f)
    
    samplingRate = d1T['data'][0][0][0][0][3][0][0]
    trialLength = TRIAL_WINDOW_LENGTH*samplingRate

    # run through all training runs
    for run in range(len(d1T['data'][0])):
        y3.append(d1T['data'][0][run][0][0][2][0]) # labels
        timestamps = d1T['data'][0][run][0][0][1][0] # timestamps
        rawData = d1T['data'][0][run][0][0][0].transpose() # chan x data

        # parse out data based on timestamps
        for start in timestamps:
            end = start + trialLength
            X3.append(rawData[:,start:end]) #15 x 2560

    del rawData
    del d1T

X3 = np.array(X3).astype(np.float32)
y3 = (np.array(y3).flatten()-1).astype(np.int64)


# rand permute dataset
idx = np.random.permutation(X3.shape[0])

X3 = X3[idx,:,:]
y3 = y3[idx]

Test subject: ['S01E.mat', 'S01T.mat']
Vehicule subjects: ['S02E.mat', 'S02T.mat', 'S03E.mat', 'S03T.mat']
Train subjects: ['S04E.mat', 'S04T.mat', 'S05E.mat', 'S05T.mat', 'S06E.mat', 'S06T.mat', 'S07E.mat', 'S07T.mat', 'S08E.mat', 'S08T.mat', 'S09E.mat', 'S09T.mat', 'S10E.mat', 'S10T.mat', 'S11E.mat', 'S11T.mat', 'S12E.mat', 'S12T.mat', 'S13E.mat', 'S13T.mat', 'S14E.mat', 'S14T.mat']


In [21]:

# if dl-eeg-playground already here, erase
!rm -rf dl-eeg-playground
!rm -rf brainDecodeSKLearnWrapper

!git clone https://github.com/NeuroTechX/dl-eeg-playground.git 
!mv dl-eeg-playground/brainDecode/towardMoabbIntegration/brainDecodeSKLearnWrapper .

Cloning into 'dl-eeg-playground'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 301 (delta 22), reused 27 (delta 11), pack-reused 261
Receiving objects: 100% (301/301), 2.19 MiB | 24.37 MiB/s, done.
Resolving deltas: 100% (150/150), done.


In [53]:
import sys
sys.path.append("brainDecode/towardMoabbIntegration")

# load the general network Trainer
from brainDecodeSKLearnWrapper.ShallowFBCSPNet_GeneralTrainer import ShallowFBCSPNet_GeneralTrainer



classifier = ShallowFBCSPNet_GeneralTrainer(nb_epoch=100)
classifier.fit(X1,y1)


Epoch 0
Train  Loss: 0.76184
Train  Accuracy: 53.2%
Test   Loss: 0.81317
Test   Accuracy: 53.6%
Epoch 1
Train  Loss: 0.78738
Train  Accuracy: 55.5%
Test   Loss: 0.87383
Test   Accuracy: 48.2%
Epoch 2
Train  Loss: 0.75034
Train  Accuracy: 53.4%
Test   Loss: 0.80308
Test   Accuracy: 50.5%
Epoch 3
Train  Loss: 0.69683
Train  Accuracy: 58.2%
Test   Loss: 0.77245
Test   Accuracy: 48.6%
Epoch 4
Train  Loss: 0.83186
Train  Accuracy: 52.0%
Test   Loss: 0.93347
Test   Accuracy: 48.6%
Epoch 5
Train  Loss: 0.66369
Train  Accuracy: 60.1%
Test   Loss: 0.74259
Test   Accuracy: 48.6%
Epoch 6
Train  Loss: 0.66870
Train  Accuracy: 59.0%
Test   Loss: 0.74957
Test   Accuracy: 51.4%
Epoch 7
Train  Loss: 0.81393
Train  Accuracy: 51.9%
Test   Loss: 0.88299
Test   Accuracy: 52.3%
Epoch 8
Train  Loss: 0.65472
Train  Accuracy: 61.4%
Test   Loss: 0.74671
Test   Accuracy: 50.5%
Epoch 9
Train  Loss: 0.62888
Train  Accuracy: 65.3%
Test   Loss: 0.70563
Test   Accuracy: 50.5%
Epoch 10
Train  Loss: 1.26972
Train  Acc

ShallowFBCSPNet_GeneralTrainer(filter_time_length=75, n_filters_spat=5,
                n_filters_time=10, nb_epoch=100, pool_time_length=60,
                pool_time_stride=30)

In [54]:
from torch import optim
from torch import nn
from braindecode.datautil.signal_target import SignalAndTarget

# define a number of train/test trials
classifier.model.conv_classifier = nn.Conv2d(5, 2,(116, 1), bias=True).cuda()

# define a number of train/test trials
nb_train_trials = int(np.floor(6/8*X2.shape[0]))

# split the dataset
train_set = SignalAndTarget(X2[:nb_train_trials], y=y2[:nb_train_trials])
test_set = SignalAndTarget(X2[nb_train_trials:], y=y2[nb_train_trials:])

train_set.X = np.concatenate((train_set.X, X3),axis=0)
train_set.y = np.concatenate((train_set.y, y3),axis=0)

print(train_set.X.shape)
print(train_set.y.shape)

#lr = 0.0001
classifier.optimizer = optim.Adam(classifier.model.parameters())

nbIter = 50

# array that tracks results
classifier.loss_rec = np.zeros((nbIter,2))
classifier.accuracy_rec = np.zeros((nbIter,2))

# run all epoch
for i_epoch in range(nbIter):

    classifier._batchTrain(i_epoch, train_set)
    classifier._evalTraining(i_epoch, train_set, test_set)

(280, 15, 3584)
(280,)
Epoch 0
Train  Loss: 0.65989
Train  Accuracy: 59.3%
Test   Loss: 0.68865
Test   Accuracy: 60.0%
Epoch 1
Train  Loss: 0.55906
Train  Accuracy: 72.5%
Test   Loss: 0.60002
Test   Accuracy: 62.5%
Epoch 2
Train  Loss: 0.47912
Train  Accuracy: 83.9%
Test   Loss: 0.53623
Test   Accuracy: 77.5%
Epoch 3
Train  Loss: 0.42306
Train  Accuracy: 88.2%
Test   Loss: 0.50652
Test   Accuracy: 82.5%
Epoch 4
Train  Loss: 0.37944
Train  Accuracy: 91.4%
Test   Loss: 0.48026
Test   Accuracy: 80.0%
Epoch 5
Train  Loss: 0.34652
Train  Accuracy: 92.9%
Test   Loss: 0.46297
Test   Accuracy: 80.0%
Epoch 6
Train  Loss: 0.31504
Train  Accuracy: 95.0%
Test   Loss: 0.44219
Test   Accuracy: 80.0%
Epoch 7
Train  Loss: 0.28979
Train  Accuracy: 95.0%
Test   Loss: 0.42785
Test   Accuracy: 80.0%
Epoch 8
Train  Loss: 0.26896
Train  Accuracy: 95.0%
Test   Loss: 0.41235
Test   Accuracy: 80.0%
Epoch 9
Train  Loss: 0.24925
Train  Accuracy: 96.8%
Test   Loss: 0.40125
Test   Accuracy: 82.5%
Epoch 10
Train  L